In [1]:
from qtools import Submitter
import pandas as pd
import numpy as np
import os
import glob

In [2]:
rbfox2_barcodes_file = '/home/iachaim/notebooks/10X/APOBEC/02_02_Subsetting_on_non_APO_overlap/results/RBFOX2-TIA1-STAMP_edits_for_RBFOX2_ORFs.csv'
tia1_barcodes_file = '/home/iachaim/notebooks/10X/APOBEC/02_02_Subsetting_on_non_APO_overlap/results/RBFOX2-TIA1-STAMP_edits_for_TIA1_ORFs.csv'

tmp_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/tmp'
bed_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_individual_barcodes_merged_bedfiles/'
bedgraph_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_individual_barcodes_merged_bedgraphs'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/outputs'

In [3]:
conf = 0.5
top = 100

In [4]:
rbfox2_barcodes = pd.read_csv(rbfox2_barcodes_file)
rbfox2_barcodes.sort_values(by=['n_edits'], ascending=False, inplace=True)
rbfox2_barcodes.reset_index(drop=True, inplace=True)
rbfox2_barcodes['rank'] = rbfox2_barcodes.index + 1
rbfox2_barcodes = rbfox2_barcodes.head(top)


tia1_barcodes = pd.read_csv(tia1_barcodes_file)
tia1_barcodes.sort_values(by=['n_edits'], ascending=False, inplace=True)
tia1_barcodes.reset_index(drop=True, inplace=True)
tia1_barcodes['rank'] = tia1_barcodes.index + 1
tia1_barcodes = tia1_barcodes.head(top)

rbfox2_barcodes.head()

,index,n_edits,RBFOX2_ORF,TIA1_ORF,feature,louvain_r0.5,rank
0,possorted_genome_bam_MD-TAACGACAGTGAGGCT-1,2.590677,0.373914,0.0,RBFOX2_ORF,Undetermined,1
1,possorted_genome_bam_MD-GCGGATCAGGTCACCC-1,2.532249,0.213277,0.0,RBFOX2_ORF,Undetermined,2
2,possorted_genome_bam_MD-CAACAACCAAGCGCTC-1,2.479191,0.222754,0.0,RBFOX2_ORF,Undetermined,3
3,possorted_genome_bam_MD-AAAGGTAGTGGATGAC-1,2.222539,0.245455,0.0,RBFOX2_ORF,Undetermined,4
4,possorted_genome_bam_MD-GGTTGTAGTAGTCCTA-1,1.909846,0.270498,0.0,RBFOX2_ORF,Undetermined,5


In [5]:
def get_sailor_bed_from_barcode_index(row, input_dir=bed_dir):
    barcode = row['index']
    bed = os.path.join(input_dir, '{}.bed'.format(barcode))
    assert os.path.exists(bed)
    return bed

In [6]:
rbfox2_barcodes['bed'] = rbfox2_barcodes.apply(get_sailor_bed_from_barcode_index, axis=1)
tia1_barcodes['bed'] = tia1_barcodes.apply(get_sailor_bed_from_barcode_index, axis=1)

rbfox2_barcodes.head()

,index,n_edits,RBFOX2_ORF,TIA1_ORF,feature,louvain_r0.5,rank,bed
0,possorted_genome_bam_MD-TAACGACAGTGAGGCT-1,2.590677,0.373914,0.0,RBFOX2_ORF,Undetermined,1,/home/bay001/projects/kris_apobec_20200121/per...
1,possorted_genome_bam_MD-GCGGATCAGGTCACCC-1,2.532249,0.213277,0.0,RBFOX2_ORF,Undetermined,2,/home/bay001/projects/kris_apobec_20200121/per...
2,possorted_genome_bam_MD-CAACAACCAAGCGCTC-1,2.479191,0.222754,0.0,RBFOX2_ORF,Undetermined,3,/home/bay001/projects/kris_apobec_20200121/per...
3,possorted_genome_bam_MD-AAAGGTAGTGGATGAC-1,2.222539,0.245455,0.0,RBFOX2_ORF,Undetermined,4,/home/bay001/projects/kris_apobec_20200121/per...
4,possorted_genome_bam_MD-GGTTGTAGTAGTCCTA-1,1.909846,0.270498,0.0,RBFOX2_ORF,Undetermined,5,/home/bay001/projects/kris_apobec_20200121/per...


In [7]:
def calculate_edit_fraction(row):
    edited, total = row['score'].split(',')
    return int(edited)/float(total)

def bed_to_bedgraph(bed, bedgraph_output_file, bed_output_file, conf):
    df = pd.read_csv(bed, sep='\t', names=['chrom','start','end','name','score','strand'])
    df['chromfx'] = 'chr' + df['chrom'].astype(str)
    df['editfrac'] = df.apply(calculate_edit_fraction, axis=1)
    df = df[df['name'] >= conf]
    df[['chromfx','start','end','name','editfrac','strand']].to_csv(bed_output_file, sep='\t', index=False, header=False)
    df[['chromfx','start','end','editfrac']].to_csv(bedgraph_output_file, sep='\t', index=False, header=False)

In [8]:
for row in rbfox2_barcodes.iterrows():
    output_file_prefix = os.path.join(bedgraph_dir, 'RBFOX2-rank-{}.{}.n_edits{}.conf{}'.format(
        row[1]['rank'], row[1]['index'], row[1]['n_edits'], conf
    ))
    bedgraph_output_file = output_file_prefix + '.bedgraph'
    bed_output_file = output_file_prefix + '.bed'
    
    bed_to_bedgraph(row[1]['bed'], bedgraph_output_file, bed_output_file, conf)
    
for row in tia1_barcodes.iterrows():
    output_file_prefix = os.path.join(bedgraph_dir, 'TIA1-rank-{}.{}.n_edits{}.conf{}'.format(
        row[1]['rank'], row[1]['index'], row[1]['n_edits'], conf
    ))
    bedgraph_output_file = output_file_prefix + '.bedgraph'
    bed_output_file = output_file_prefix + '.bed'
    
    bed_to_bedgraph(row[1]['bed'], bedgraph_output_file, bed_output_file, conf)